In [1]:
## Install all dependencies
%pip install gensim
# %pip install nltk
# %pip install pandas
# %pip install numpy
%pip install requests
%pip install PyPDF2

     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     --------------------------------------- 0.0/23.3 MB 163.8 kB/s eta 0:02:22
     --------------------------------------- 0.0/23.3 MB 163.8 kB/s eta 0:02:22
     --------------------------------------- 0.0/23.3 MB 163.4 kB/s eta 0:02:23
     --------------------------------------- 0.1/23.3 MB 231.0 kB/s eta 0:01:41
     --------------------------------------- 0.1/23.3 MB 327.2 kB/s eta 0:01:11
     --------------------------------------- 0.2/23.3 MB 398.2 kB/s eta 0:00:59
     --------------------------------------- 0.2/23.3 MB 478.0 kB/s eta 0:00:49
     --------------------------------------- 0.2/23.3 MB 514.3 kB/s eta 0:00:45
      -------------------------------------- 0.3/23.3 MB 593.9 kB/s eta 0:00:39
      -------------------------------------- 0.4/23.3 MB 636.5 k


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install nltk
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
#from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
#import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import json

# Read the JSONL file and extract labels
majors_list = []

with open('majors.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        label_parts = data["label"].split('|')
        
        # Extract the label after the second '|', replace hyphens with spaces
        extracted_label = label_parts[2].replace('-', ' ')
        
        # Append the extracted label to the list
        majors_list.append(extracted_label)

# Print the resulting list
majors_list = list(set(majors_list))
print(majors_list)


['data science', 'information science', 'computer science', 'software project management', 'cybersecurity', 'data analysis', 'artificial intelligence', 'computer vision', 'computer software', 'systems analysis', 'programming', 'mathematics', 'web development', 'statistics', 'software development', 'nlp', 'software engineering', 'systems and network administration', 'network security', 'information systems', 'information technology', 'computer engineering', 'nlp engineering', 'computer programming', 'data engineering', 'ai engineering']


In [7]:
skills_list = []

with open('skills.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        label_parts = data["label"].split('|')
        
        # Extract the label after the second '|', replace hyphens with spaces
        extracted_label = label_parts[1].replace('-', ' ')
        
        # Append the extracted label to the list
        skills_list.append(extracted_label)

# Print the resulting list
skills_list = list(set(skills_list))
print(skills_list)

['mvc tools', 'spring', 'vue.js', 'data analysis', 'apache zeppelin', 'file uploads', 'random access', 'synthetic aperture radar', 'phpunit', 'image analysis api', 'neural coding', 'correctness', 'api tools', 'git', 'query optimization', 'confluence', 'metabase', 'information theory', 'faceted classification', 'hugo', 'classifier linguistics', 'semantic computing', 'jupyter notebook', 'analytics', 'genetic algorithm', 'abstract machine', 'segmentation', 'g suite', 'facial expression detection', 'backbone.js', 'critical mass software engineering', 'pattern recognition', 'gluon', 'data stores', 'error concealment', 'messenger platform', 'simulated annealing', 'virtualization platform', 'wireframing', 'utilities', 'brute force search', 'fragmentation computing', 'gatling', 'read write memory', 'latex', 'opengl', 'clicktale', 'survey widget', 'edit distance', 'semantics', 'augmented reality', 'content writing', 'dynamic loading', 'rocket', 'amazon ec2', 'biomedical engineering', 'collision

In [8]:
degrees_list = []

with open('degrees.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        label_parts = data["label"].split('|')
        
        # Extract the label after the second '|', replace hyphens with spaces
        extracted_label = label_parts[1].replace('-', ' ')
        
        # Append the extracted label to the list
        degrees_list.append(extracted_label)

# Print the resulting list
degrees_list = list(set(degrees_list))
print(degrees_list)

['BS LEVEL', 'PHD LEVEL', 'MS LEVEL']


In [9]:
skills_degrees_majors = degrees_list + skills_list + majors_list

In [ ]:
skills_degrees_majors

In [11]:
# Tag data - labels every skill/degree/major with a unique identifier - required for doc2vec model
data = list(skills_degrees_majors)
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [13]:
# Model initialization
model = Doc2Vec(vector_size = 50,
epochs = 40,
alpha = 0.001
)

In [14]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

128


In [15]:
tagged_data

[TaggedDocument(words=['bs', 'level'], tags=['0']),
 TaggedDocument(words=['phd', 'level'], tags=['1']),
 TaggedDocument(words=['ms', 'level'], tags=['2']),
 TaggedDocument(words=['mvc', 'tools'], tags=['3']),
 TaggedDocument(words=['spring'], tags=['4']),
 TaggedDocument(words=['vue.js'], tags=['5']),
 TaggedDocument(words=['data', 'analysis'], tags=['6']),
 TaggedDocument(words=['apache', 'zeppelin'], tags=['7']),
 TaggedDocument(words=['file', 'uploads'], tags=['8']),
 TaggedDocument(words=['random', 'access'], tags=['9']),
 TaggedDocument(words=['synthetic', 'aperture', 'radar'], tags=['10']),
 TaggedDocument(words=['phpunit'], tags=['11']),
 TaggedDocument(words=['image', 'analysis', 'api'], tags=['12']),
 TaggedDocument(words=['neural', 'coding'], tags=['13']),
 TaggedDocument(words=['correctness'], tags=['14']),
 TaggedDocument(words=['api', 'tools'], tags=['15']),
 TaggedDocument(words=['git'], tags=['16']),
 TaggedDocument(words=['query', 'optimization'], tags=['17']),
 Tagged

In [16]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/40
Training epoch 2/40
Training epoch 3/40
Training epoch 4/40
Training epoch 5/40
Training epoch 6/40
Training epoch 7/40
Training epoch 8/40
Training epoch 9/40
Training epoch 10/40
Training epoch 11/40
Training epoch 12/40
Training epoch 13/40
Training epoch 14/40
Training epoch 15/40
Training epoch 16/40
Training epoch 17/40
Training epoch 18/40
Training epoch 19/40
Training epoch 20/40
Training epoch 21/40
Training epoch 22/40
Training epoch 23/40
Training epoch 24/40
Training epoch 25/40
Training epoch 26/40
Training epoch 27/40
Training epoch 28/40
Training epoch 29/40
Training epoch 30/40
Training epoch 31/40
Training epoch 32/40
Training epoch 33/40
Training epoch 34/40
Training epoch 35/40
Training epoch 36/40
Training epoch 37/40
Training epoch 38/40
Training epoch 39/40
Training epoch 40/40
Model saved


In [5]:
%pip install pdfplumber



     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     ---------------------------------------- 0.1/5.6 MB 812.7 kB/s eta 0:00:07
      --------------------------------------- 0.1/5.6 MB 655.4 kB/s eta 0:00:09
     - -------------------------------------- 0.2/5.6 MB 833.5 kB/s eta 0:00:07
     - -------------------------------------- 0.2/5.6 MB 926.0 kB/s eta 0:00:06
     - -------------------------------------- 0.2/5.6 MB 808.4 kB/s eta 0:00:07
     -- ------------------------------------- 0.3/5.6 MB 883.3 kB/s eta 0:00:07
     -- ------------------------------------- 0.3/5.6 MB 936.6 kB/s eta 0:00:06
     -- ------------------------------------- 0.3/5.6 MB 832.3 kB/s eta 0:00:07
     --- ------------------------------------ 0.5/5.6 MB 1.0 MB/s eta 0:00:05
     ---- ----------------------------------- 0.6/5.6 MB 1.2 MB/s eta 0:00:05
     ----- ---------------------------------- 0.7/5.6 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
# pdf = PyPDF2.PdfReader('./kainat.pdf')
# resume = ""
# for i in range(len(pdf.pages)):
#     pageObj = pdf.pages[i]
#     resume += pageObj.extract_text()

In [6]:
import pdfplumber

resume = ""
with pdfplumber.open('./CV_sample.pdf') as pdf:
    for page in pdf.pages:
        resume += page.extract_text()

In [7]:
resume

"Nisa Nadeem\nLahore, Pakistan nisanadeem90@gmail.com +92 3094283955 in/nisa-nadeem-16aa08227\nSUMMARY\nAn independent learner, detail-oriented and a team player with fundamental knowledge of\nsoftware design and development in creating and executing innovative software solution in a\ndeadline-driven environment to enhance business productivity.\nEXPERIENCE\nStudent Ambassador|Tkxel|Pakistan, Lahore|August 2023- Present\nAs a Student Ambassador at Tkxel, I e\x00ectively represented the organization's values and engaged with diverse audiences, showcasing the\ncompany's innovations and fostering meaningful connections within the student community.\nTeaching Assistant|FAST NUCES Lahore|January 2022 - June 2023\nContent Writer|www.pakzilla.com/|Hybrid|August 2021 - December 2021\nI honed my passion for technology and coding into engaging and informative content as I crafted articles for a tech-focused blog, delivering\ncomplex concepts in an accessible and reader-friendly manner.\nPROJECTS

In [8]:
# JD by input text:
#jd = "Project Manager BS Computer Science/Software Engineering/Information Technology/Software Project Management What you'll do  Work closely with engineers, product managers, designers, researchers, business, operations, support and other key leaders to conceptualize cross-functional initiatives with layers of dependencies, and plan their execution.Focus areas for your programs will include building new product features, expanding platform capability, overall system scalability & reliability, and tooling enhancements that will improve a customer's experience.  Have a solid understanding of the architecture to exercise technical judgment as necessary  Set accountability through communication of progress transparency and visibility on the next set of actions and milestones  Manage resource allocation and risks to optimize delivery  You will manage expectations for all stakeholders from leadership to individual teams, possess strong problem solving and negotiation skills, and earn trust of partners and teams.  You understand engineers and enjoy working with them, building trust by developing strong technical domain knowledge that adds value beyond project management.  You’ll develop processes to ensure teamwork is streamlined to manage complexity and optimized velocity. You are thoughtful about how and when to implement processes with a light touch and clear understanding of success with Abyan Capital  Continuously look for opportunities for optimizing pace and quality  You’ll balance urgency and priority while moving the needle on multiple initiatives."
#jd ="iOS Developer  BS Computer Science/Software Engineering/Information Technology Looking for IOS Developer having to 3+ years of experience. Requirements: • Bachelor’s degree in Computer Science or Software Engineering. • Proven experience as an app developer. • Proficient in Objective-C, Swift, and Cocoa Touch. • Extensive experience with iOS Frameworks such as Core Data and Core Animation. • Knowledge of iOS back-end services. • Knowledge of Apple’s design principles and application interface guidelines. • Proficient in code versioning tools including Mercurial, Git, and SVN. • Knowledge of C-based libraries. • Familiarity with push notifications, APIs, and cloud messaging. • Experience with continuous integration."
jd = "Php – Laravel Developer BS Computer Science/Software Engineering/Information Technology Manafa tech requires a talented “PHP Developer”.  You must have very strong programming skills, specially in open source technologies (PHP Core and Frameworks CI, YII, Laravel, CakePHP, Zend) , JS libraries.  Must be graduate or master in computer sciences from reputable university  Personal Characteristics: Ability to work quickly but with a strong attention to detail and accuracy Team player and good interpersonal skills Ability to perform under stress and aggressive deadlines Self-organized with the ability to plan ahead Punctual and organized  Tools and Technologies : • PHP Core and Frameworks CI, YII, Laravel, CakePHP, Zend, Symphony • MySQL • JavaScript, Angular.js, JQuery, Node.js"
#jd = "Graphic Designer-Ui/Ux Designer- Frontend Developer/designer The ideal candidates will have strong creative skills and a portfolio of work which demonstrates their passion. Bachelors BS Computer Science/Software Engineering/Information Technology    Skills Required: Photoshop, Illustrator/Corel Draw, AdobeXD, Figma and others within Adobe Creative Cloud Suite.  First-hand knowledge of: UX Design: An understanding/appreciation for UX Basic Coding: An understanding of how creative sits within digital platforms such as: HTML, CSS and JavaScript A good understanding of mobile first design  The ideal candidates will have strong creative skills and a portfolio of work which demonstrates their passion.  Basic Coding: An understanding of how creative sits within digital platforms such as: HTML, CSS and JavaScript A good understanding of mobile first design  Photoshop, Illustrator/Corel Draw, AdobeXD, Figma and others within Adobe Creative Cloud Suite. First-hand knowledge of: UX Design: An understanding/appreciation for UX"

In [9]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) #removes non-printable characters
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    # Remove numerical values from the text
    resumeText = re.sub(r'\d+', '', resumeText)

    return resumeText.lower()

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\]'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\]'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5756\1993080037.py:2: SyntaxWarning: invalid escape sequence '\S'
  resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5756\1993080037.py:4: SyntaxWarning: invalid escape sequence '\S'
  resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5756\1993080037.py:5: SyntaxWarning: invalid escape sequence '\S'
  resumeText = re.sub('@\S+', '  '

In [10]:
import spacy
from spacy.lang.en import English

In [11]:
def match_majors_by_spacy(doc):            
        nlp = English()
        patterns_path = "majors.jsonl"
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        
        acceptable_majors_list = []
        
       
        doc1 = nlp(doc)
        acceptable_majors = []
        for ent in doc1.ents:
                labels_parts = ent.label_.split('|')
                if labels_parts[0] == 'MAJOR':
                    if labels_parts[2].replace('-', ' ') not in acceptable_majors:
                        acceptable_majors.append(labels_parts[2].replace('-', ' '))
                    if labels_parts[2].replace('-', ' ') not in acceptable_majors:
                        acceptable_majors.append(labels_parts[2].replace('-', ' '))
        if acceptable_majors:
            acceptable_majors_list.append(', '.join(acceptable_majors))
        
        return acceptable_majors_list
def match_degrees_by_spacy(doc):
        nlp = English()
        patterns_path = "degrees.jsonl"
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        acceptable_degrees_list = []
        
        
        doc1 = nlp(doc)
        degree_levels = []
        for ent in doc1.ents:
                labels_parts = ent.label_.split('|')
                if labels_parts[0] == 'DEGREE':
                    if labels_parts[1] not in degree_levels:
                        degree_levels.append(labels_parts[1])
        if degree_levels:
                acceptable_degrees_list.append(', '.join(degree_levels))

        return acceptable_degrees_list

def match_skills_by_spacy(doc):
        nlp = English()
        patterns_path = "skills.jsonl"
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        acceptable_skills_list = []
        
        
        doc1 = nlp(doc)
        job_skills = []
        for ent in doc1.ents:
                labels_parts = ent.label_.split('|')
                if labels_parts[0] == 'SKILL':
                    if labels_parts[1].replace('-', ' ') not in job_skills:
                        job_skills.append(labels_parts[1].replace('-', ' '))
        if job_skills:
                acceptable_skills_list.append(', '.join(job_skills))

        return acceptable_skills_list
        

In [12]:
cv_majors = match_majors_by_spacy(cleanResume(resume))
cv_degrees = match_degrees_by_spacy(cleanResume(resume))
cv_skills = match_skills_by_spacy(cleanResume(resume))

cv_majors = [major.strip() for majors in cv_majors for major in majors.split(',')]
cv_skills = [major.strip() for majors in cv_skills for major in majors.split(',')]
cv_degrees = [major.strip() for majors in cv_degrees for major in majors.split(',')]

In [13]:
cv_all = cv_majors + cv_degrees + cv_skills

cv_all = list(set(cv_all))

In [147]:
#cv_all = ' '.join(cv_all)

In [14]:
cv_all

['backend',
 'html',
 'software engineering',
 'business',
 'react',
 'css',
 'software',
 'frontend',
 'design',
 'expressjs',
 'database',
 'BS-LEVEL',
 'java',
 'react native',
 'javascript',
 'mysql',
 'spring boot',
 'microsoft sql server',
 'content writing',
 'node.js',
 'code',
 'spring',
 'c']

In [15]:
jd_majors = match_majors_by_spacy(cleanResume(jd))
jd_degrees = match_degrees_by_spacy(cleanResume(jd))
jd_skills = match_skills_by_spacy(cleanResume(jd))

jd_majors = [major.strip() for majors in jd_majors for major in majors.split(',')]
jd_skills = [major.strip() for majors in jd_skills for major in majors.split(',')]
jd_degrees = [major.strip() for majors in jd_degrees for major in majors.split(',')]

jd_all = jd_majors + jd_degrees + jd_skills
jd_all = list(set(jd_all))


In [ ]:
#jd_all = ' '.join(jd_all)

In [16]:
jd_all

['javascript',
 'yii',
 'software engineering',
 'mysql',
 'zend framework',
 'operations research',
 'cakephp',
 'programming',
 'node.js',
 'frameworks',
 'information technology',
 'angular',
 'MS-LEVEL',
 'BS-LEVEL',
 'php',
 'jquery',
 'computer science',
 'laravel']

In [17]:
# Model evaluation
from sklearn.metrics.pairwise import cosine_similarity

model = Doc2Vec.load('cv_job_maching.model')

In [17]:

# v1 = model.infer_vector(cv_all.split())
# v2 = model.infer_vector(jd_all.split())
# similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))

In [18]:
v1 = []
v2 = []

for element in cv_all:
    v1.append(model.infer_vector(element.split()))

for element in jd_all:
    v2.append(model.infer_vector(element.split()))

In [19]:
similarity = 0
i = 0
tot = 0
if v1 != []:
    for element in v2:
        if jd_all[i] in cv_all:
            similarity = similarity + 1
            tot = tot + 1
        else:
            max = -1
            for cv_item in v1:
                newsimilarity = (np.dot(np.array(cv_item), np.array(element))) / (norm(np.array(cv_item)) * norm(np.array(element)))
                if newsimilarity > max:
                    max = newsimilarity
            #print(max)
            if max >= 0.3:
                similarity = similarity + max
                tot = tot + 1
        # print(similarity)
        #print(jd_all[i])
        i=i+1
        
    if tot < len(jd_all)/2:
        tot = len(jd_all)
    
    similarity = round(similarity/tot*100,3)

In [20]:
similarity

68.636

In [106]:
# from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# from gensim.models import Word2Vec
# from gensim.models import TfidfModel
# from gensim.corpora import Dictionary
# from nltk.tokenize import word_tokenize
# import numpy as np
# from numpy.linalg import norm

In [155]:
# def semantic_similarity_word2vec(job, resume, word2vec_model):
#     # Tokenize the job and resume
#     job_tokens = word_tokenize(job.lower())
#     resume_tokens = word_tokenize(resume.lower())

#     job_tokens= [token for token in job_tokens if token != ',']
#     resume_tokens = [token for token in resume_tokens if token != ',']

    
    
#     # Get the vectors for each token in the job and resume
#     job_vectors = [word2vec_model.wv[word] for word in job_tokens if word in word2vec_model.wv]
#     resume_vectors = [word2vec_model.wv[word] for word in resume_tokens if word in word2vec_model.wv]

#     # Check if there are vectors available
#     # if job_vectors and resume_vectors:
#     #     # Average of vectors
#     #     job_vector = np.mean(job_vectors, axis=0)
#     #     resume_vector = np.mean(resume_vectors, axis=0)

#     #     # Calculate cosine similarity
#     #     similarity = 100 * np.dot(job_vector, resume_vector) / (norm(job_vector) * norm(resume_vector))

#     #     return round(similarity, 3)
#     # else:
#     #     return "No vectors found for one or both of the documents."
#     score = 0
#     for i in range(len(job_vectors)):
        
#         if any(np.isin(elem, resume_vectors) for elem in job_vectors[i]):
#             score += 1
#         else: # compares 1 word of JD with ALL CV words....jiski maximum cosine similarity..uska score uthata hai
#             print("printing max = ")
#             print(max(cosine_similarity([job_vectors[i]],resume_vectors)[0]))
#             if max(cosine_similarity([job_vectors[i]],resume_vectors)[0]) >= 0.15:
#                 score += max(cosine_similarity([job_vectors[i]],resume_vectors)[0])
        
#         print(job_tokens[i])
#         print(score)
#     score = score/len(job_vectors)  
#     return round(score,3)

In [171]:
# # Example with one job description and one resume
# job_description = jd_all
# resume = cv_all

# # Load your pre-trained Word2Vec model
# word2vec_model = Word2Vec.load('./word2vec_model.model')  # Replace with the path to your saved model

# similarity_score = semantic_similarity_word2vec(job_description, resume, word2vec_model)
# print(f"Similarity between Job and Resume: {similarity_score}")

printing max = 
0.17867742
computer
0.17867742478847504
printing max = 
0.119518474
science
0.17867742478847504
printing max = 
0.16964702
software
0.3483244478702545
engineering
1.3483244478702545
information
2.3483244478702545
printing max = 
0.1948782
technology
2.54320265352726
printing max = 
0.17867742
bs-level
2.721880078315735
printing max = 
0.119518474
graphic
2.721880078315735
printing max = 
0.16964702
design
2.8915271013975143
user
3.8915271013975143
printing max = 
0.1948782
interface
4.08640530705452
printing max = 
0.14729396
frontend
4.08640530705452
printing max = 
0.25795498
computer
4.344360291957855
Similarity between Job and Resume: 0.334
